# Conexion

In [3]:
# Variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables desde el archivo .env
load_dotenv()

# Obtener las variables desde el entorno
account_name = os.getenv("VTEX_ACCOUNT_NAME")
app_key = os.getenv("VTEX_APP_KEY")
app_token = os.getenv("VTEX_APP_TOKEN")

# Validar que las variables se cargaron correctamente
if not all([account_name, app_key, app_token]):
    raise ValueError("Faltan credenciales en el archivo .env")

In [4]:
# Librerias
# import sys
# sys.path.append(r"C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages")
import requests # peticiones HTML
import pandas as pd # manejo de datos
import matplotlib.pyplot as plt # graficacion

In [5]:
# Configuración de la URL y headers
base_url = f"https://{account_name}.vtexcommercestable.com.br/api/oms/pvt/orders"
headers = {
    "X-VTEX-API-AppKey": app_key,
    "X-VTEX-API-AppToken": app_token,
    "Content-Type": "application/json",
}

In [10]:
# Parámetros para filtrar órdenes - Se puede obtener de VTEX, compartiendo filtros
params = {
    "date":
      {
        # "f_creationDate": "custom:[2023-12-31T03:00:00.000Z TO 2024-12-31T02:59:59.999Z]"
        # "f_creationDate": "current-year",
        "f_creationDate": "current-month"
      },
    "page": 1, # de 1 a 30
    "per_page": 100, # de 1 a 100
}

In [ ]:
# Función llamada a la API
def get_orders():
  orders = []
  page = 1
  while True:    
    params['page'] = page
    response = requests.get(base_url, headers=headers, params=params)
    
    # Verificar si la respuesta es válida
    try:
      data = response.json()
    except ValueError as e:
      print(f"Error al procesar la respuesta JSON: {response.text}")
      break

    # Asegurarse de que "list" exista en la respuesta
    if "list" not in data:
      print(f"Error: 'list' no encontrado en la respuesta: {data}")
      break
    
    orders_list = data["list"]
    
    if not orders_list:
      break
    
    for order in orders_list:
      order_data = {
        'sequence': order.get('sequence', ''),
        'creationDate': order.get('creationDate', ''),
        'totalItems': order.get('totalItems', 0),
        'totalValue': order.get('value', 0),
        'paymentNames': ', '.join([payment.get('paymentSystem', '') for payment in order.get('payments', [])]),
        'statusDescription': order.get('statusDescription', ''),
        'hostname': order.get('hostname', ''),
        'origin': order.get('origin', '')
      }
      orders.append(order_data)
    
    if not data:
      break
          
    page += 1
  return orders

# Obtener los pedidos
orders = get_orders()

# Convertir los pedidos a un DataFrame de pandas
df_orders = pd.DataFrame(orders)

# Guardar el DataFrame a un archivo CSV
df_orders.to_csv('pedidos_2024.csv', index=False)

print("Datos de pedidos guardados en 'pedidos_2024.csv'")

In [ ]:
# Realizar peticion


response = requests.get(base_url, headers=headers, params=params)

response

In [16]:
# PARA IMPRIMIR TXT RESULTADO
# ordersTxt = response.json()

# Guardar la respuesta en un archivo .txt
# with open("respuesta_ordenes.txt", "w") as file:
#    file.write(str(ordersTxt))  # Convertir el diccionario a una cadena de texto

# Este código guardará la respuesta en un archivo llamado "respuesta_ordenes.txt"

In [8]:
# SE ARMA DATA FRAME
df = pd.DataFrame()
columnas = ["sequence", "creationDate", "totalItems", "totalValue", "paymentNames", "status", "hostname", "origin"]

# Procesar los datos de la respuesta
if response.status_code == 200:
    orders = response.json()["list"]  # Extraemos la lista de órdenes

    # Verificar si hay órdenes
    if orders:
        # Crear un DataFrame a partir de las órdenes
        dfOrig = pd.DataFrame(orders)
        
        # Filtrar las columnas relevantes
        df = dfOrig[columnas]
        
        # Convertir la columna 'creationDate' a tipo datetime sin zona horaria
        df.loc[:, "creationDate"] = pd.to_datetime(df["creationDate"]).dt.strftime('%Y-%m-%d %H:%M:%S')  # Solo mostrar fecha y hora, sin la zona horaria
        
        # Convertir 'totalValue' de centavos a la moneda original (asumiendo que el valor está en centavos)
        df.loc[:, "totalValue"] = df["totalValue"] / 100
        
    else:
        print("No se encontraron órdenes.")
else:
    print(f"Error al obtener datos: {response.status_code}")
    print("Detalles del error:", response.text)
    
# Mostrar las primeras filas para verificar los datos
df.head()

,sequence,creationDate,totalItems,totalValue,paymentNames,status,hostname,origin
0,2216780,2024-12-30 17:18:29,1,123200.0,,window-to-cancel,newsport,Fulfillment
1,2216774,2024-12-30 17:15:32,1,120999.0,Naranja,window-to-cancel,newsport,Marketplace
2,2216770,2024-12-30 17:14:45,1,149999.0,Visa,ready-for-handling,newsport,Marketplace
3,2216768,2024-12-30 17:14:25,1,38999.0,GOcuotas,window-to-cancel,cero26,Marketplace
4,2216766,2024-12-30 17:12:50,2,189998.0,American Express,ready-for-handling,newsport,Marketplace


In [9]:
# se exporta a CSV
df.to_csv("ventas.csv", index=False)